# Lecture 60: Activity recognition using CNN-LSTM
## 60a: Train CNN for activity classification

#### Note:
      1. Run lecture59_preProc1.ipynb before running executing this notebook
      2. Files lecture60a.ipynb, lecture60b.ipynb, lecture60c.ipynb are part of the same tutorial and are to be exeuted sequentially  
#### Dataset: [UCF101](https://www.crcv.ucf.edu/research/data-sets/ucf101/)

In [ ]:
%matplotlib inline
import copy
import time
import torch
import numpy as np
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
import matplotlib.pyplot as plt
from torchvision import transforms,datasets, models

print(torch.__version__) # This code has been updated for PyTorch 1.0.0

In [ ]:
# Check availability of GPU

use_gpu = torch.cuda.is_available()
# use_gpu = False # Uncomment in case of GPU memory error
if use_gpu:
    print('GPU is available!')
    device = "cuda"
    pinMem = True
else:
    print('GPU is not available!')
    device = "cpu"
    pinMem = False

In [ ]:
# Loading data from folder using ImageFolder
trainDir = 'train_5class'
valDir = 'test_5class'
apply_transform = transforms.Compose([
        transforms.Resize(256),
        transforms.CenterCrop(224),
        transforms.ToTensor()])

BatchSize = 128
# Training dataloader
train_dataset = datasets.ImageFolder(trainDir,transform=apply_transform)
trainLoader = torch.utils.data.DataLoader(train_dataset, batch_size=BatchSize, shuffle=True,num_workers=4, pin_memory=pinMem)

# Test dataloader
test_dataset = datasets.ImageFolder(valDir,transform=apply_transform)
testLoader = torch.utils.data.DataLoader(test_dataset, batch_size=BatchSize, shuffle=False,num_workers=4, pin_memory=pinMem)

In [ ]:
# Size of train and test datasets
print('No. of samples in train set: '+str(len(trainLoader.dataset)))
print('No. of samples in test set: '+str(len(testLoader.dataset)))

## Define network architecture

In [ ]:
net = models.resnet18(pretrained=True)
print(net)

In [ ]:
# Counting number of trainable parameters
totalParams = 0
for name,params in net.named_parameters():
    print(name,'-->',params.size())
    totalParams += np.sum(np.prod(params.size()))
print('Total number of parameters: '+str(totalParams))

In [ ]:
# Modifying the last fully-connected layer for 5 classes
net.fc = nn.Linear(512,5) 

In [ ]:
net = net.to(device)

## Define loss function and optimizer

In [ ]:
criterion = nn.NLLLoss() # Negative Log-likelihood
optimizer = optim.Adam(net.fc.parameters(), lr=1e-4) # Adam

## Train the network

In [ ]:
iterations = 10

trainLoss = []
trainAcc = []
testLoss = []
testAcc = []

start = time.time()
for epoch in range(iterations):
    epochStart = time.time()
    runningLoss = 0.0   
    avgTotalLoss = 0.0
    running_correct = 0   
    
    net.train() # For training 
    batchNum = 1
    for data in trainLoader:
        inputs,labels = data        
        inputs, labels = inputs.to(device), labels.to(device)
        outputs = net(inputs)
        _, predicted = torch.max(outputs.data, 1)
        running_correct += (predicted == labels.data).sum()            
       
        # Initialize gradients to zero
        optimizer.zero_grad()             
        
        # Compute loss/error
        loss = criterion(F.log_softmax(outputs,dim=1), labels)
        # Backpropagate loss and compute gradients
        loss.backward()
        # Update the network parameters
        optimizer.step()
        # Accumulate loss per batch
        runningLoss += loss.item() 
        batchNum += 1

    avgTrainAcc = 100*float(running_correct)/float(len(trainLoader.dataset))
    avgTrainLoss = runningLoss/(float(len(trainLoader.dataset))/BatchSize)  
    trainAcc.append(avgTrainAcc)
    trainLoss.append(avgTrainLoss)  
    
    # Evaluating performance on test set for each epoch
    net.eval() # For testing [Affects batch-norm and dropout layers (if any)]
    running_correct = 0 
    with torch.no_grad():
        for data in testLoader:
            inputs,labels = data
            
            inputs, labels = inputs.to(device), labels.to(device)                
            outputs = net(inputs)
            _, predicted = torch.max(outputs.data, 1)
            running_correct += (predicted == labels.data).sum()
        
            loss = criterion(F.log_softmax(outputs,dim=1), labels)
        
            runningLoss += loss.item()   

    avgTestLoss = runningLoss/(float(len(testLoader.dataset))/BatchSize)
    avgTestAcc = 100*float(running_correct)/float(len(testLoader.dataset))
    testAcc.append(avgTestAcc)  
    testLoss.append(avgTestLoss)
    
    # Plotting training loss vs Epochs
    fig1 = plt.figure(1)        
    plt.plot(range(epoch+1),trainLoss,'r-',label='train')  
    plt.plot(range(epoch+1),testLoss,'g-',label='test') 
    if epoch==0:
        plt.legend(loc='upper left')
        plt.xlabel('Epochs')
        plt.ylabel('Loss')   
    # Plotting testing accuracy vs Epochs
    fig2 = plt.figure(2)        
    plt.plot(range(epoch+1),trainAcc,'r-',label='train')    
    plt.plot(range(epoch+1),testAcc,'g-',label='test')        
    if epoch==0:
        plt.legend(loc='upper left')
        plt.xlabel('Epochs')
        plt.ylabel('Accuracy')  
          
        
    epochEnd = time.time()-epochStart
    print('Iteration: {:.0f} /{:.0f};  Training Loss: {:.6f} ; Training Acc: {:.3f}'\
          .format(epoch + 1,iterations,avgTrainLoss,avgTrainAcc))
    print('Iteration: {:.0f} /{:.0f};  Testing Loss: {:.6f} ; Testing Acc: {:.3f}'\
          .format(epoch + 1,iterations,avgTestLoss,avgTestAcc))
   
    print('Time consumed: {:.0f}m {:.0f}s'.format(epochEnd//60,epochEnd%60))
end = time.time()-start
print('Training completed in {:.0f}m {:.0f}s'.format(end//60,end%60))


### Save trained model

In [ ]:
torch.save(net.state_dict(), 'resnet18Pre_fcOnly5class_ucf101_10adam_1e-4_b128.pt')